<div align="center" style="">

# Zadanie 3

<div>

In [1]:
import numpy as np
import random
from typing import List

CHROMOSOME_LEN = 20
POPULATION_LEN = 3
ITERATION_NUM = 100

def binary_random(p):
    return 1 if random.random() <= p else 0

def random_individual(p: List[float]) -> np.ndarray:
    return np.array([binary_random(pi) for pi in p])

def random_population(p: List[float], N: int = POPULATION_LEN) -> np.ndarray:
    return np.array([random_individual(p) for _ in range(N)])

def population_evaluation(P: List[List[int]], F):
    return np.array([F(x) for x in P])

def population_based_incremental_learning(F, N: int, O1, O2, O3):
    pro_vector = np.array([0.5 for _ in range(N)])                      # Initial Probabilty Vector
    ran_population = random_population(pro_vector, POPULATION_LEN)      # Initial Random Population 
    fitness = population_evaluation(ran_population, F)                  # Results
    best_individual = []
    best_score = 0
    for _ in range(ITERATION_NUM):
        x = ran_population[np.argmax(fitness)]

        if best_score < fitness[np.argmax(fitness)]:
            best_score = fitness[np.argmax(fitness)]
            best_individual = x

        for i in range(N):
            pro_vector[i] = pro_vector[i] * (1 - O1) + (x[i] * O1)

        for i in range(N):
            if random.random() <= O2:
                pro_vector[i] = pro_vector[i] * (1 - O3) + O3 * binary_random(0.5)

        ran_population = random_population(pro_vector, POPULATION_LEN)
        fitness = population_evaluation(ran_population, F)

    return best_individual, best_score


In [2]:
from collections import Counter

raws = []
expert = []
classification = []

with open('lista01/ImageRawReduced.txt', "r") as f:
    for line in f.readlines():
        raws.append(line.split())

with open('lista01/ImageExpertReduced.txt', "r") as f:
    for line in f.readlines():
        expert.append(line.split())

with open ('lista01/ClassificationRules.txt', "r") as f:
    for line in f.readlines():
        classification.append(line.split())

In [ ]:
def func(seq: List[int]) -> float:
    n = len(seq)
    m = len(classification[0])
    res = 0

    for p in range(m):
        c = Counter()

        c[classification[seq != 0][p]] += 1

        if c.most_common(1)[0][0] == expert[0][p]:
            res += 1

    return res / m
        
x, s = population_based_incremental_learning(func, 266, 0.1, 0.1, 0.1)
print(f'{x}: {(s * 100):.02f}')

[1 0 1 0 0 1 1 0 1 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1
 1 1 0 1 1 1 1 0 1 1 1 1 0 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 1 0
 1 0 0 1 0 0 1 0 1 1 1 0 0 0 0 1 1 0 0 1 0 1 1 1 1 1 0 1 1 1 1 0 1 0 1 0 0
 1 1 0 1 1 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 1 1 0 1 0 1 1 1 1 0 1 0
 1 1 1 1 1 0 0 1 1 0 0 1 0 0 0 0 1 0 1 1 0 0 1 1 1 1 0 1 0 1 1 1 0 1 1 0 0
 1 0 1 0 0 0 0 0 1 1 0 0 1 1 1 0 0 0 0 1 1 1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0
 1 0 0 1 0 1 0 1 1 1 1 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 1
 0 1 1 0 0 0 0]: 98.02


## ChatGPT

In [4]:
# import numpy as np

# # ---------- SZYBKIE WCZYTANIE DANYCH ----------
# def load_data(paths):
#     rules_path, expert_path = paths

#     # 1) Wczytaj jako float (działa dla "1.0000000e+000"), potem zaokrąglij -> int
#     expert_f = np.loadtxt(expert_path, dtype=np.float32)       # (9350,) lub (1,9350)
#     expert   = np.rint(expert_f).astype(np.int16).ravel()      # 1..3

#     rules_f  = np.loadtxt(rules_path, dtype=np.float32)        # (266,9350) lub (9350,266)
#     rules    = np.rint(rules_f).astype(np.int16)

#     # 2) Orientacja: chcemy (R x M) = (liczba_reguł x liczba_punktów)
#     if rules.shape[1] != expert.shape[0] and rules.shape[0] == expert.shape[0]:
#         rules = rules.T

#     R, M = rules.shape
#     assert M == expert.shape[0], f"Niezgodne wymiary: rules {rules.shape}, expert {expert.shape}"

#     # 3) Bezpieczne sprowadzenie do 0..2
#     if not np.all(np.isin(expert, [1,2,3])):
#         # jeżeli trafi się np. 0 lub 4 przez błąd zapisu - przytnij i ostrzeż
#         expert = np.clip(expert, 1, 3)
#     expert0 = (expert - 1).astype(np.uint8)

#     rules_ok = rules.copy()
#     if not np.all(np.isin(rules_ok, [1,2,3])):
#         rules_ok = np.clip(rules_ok, 1, 3)
#     rules0 = (rules_ok - 1).astype(np.uint8)  # (R,M) w 0..2

#     # 4) Prekomputacja one-hot: OH[r,k,m] = 1 gdy reguła r przewiduje klasę k dla punktu m
#     OH = np.zeros((R, 3, M), dtype=np.uint8)
#     for k in range(3):
#         OH[:, k, :] = (rules0 == k)

#     # szybki sanity-check
#     # print("expert0:", expert0.shape, "OH:", OH.shape)  # (9350,), (266,3,9350)

#     return expert0, OH



# # ---------- BARDZO SZYBKA EWALUACJA POPULACJI ----------
# def fitness_population(pop, OH, expert0):
#     """
#     pop: bool/0-1, shape (P, R) lub (R,)
#     OH:  uint8,   shape (R, 3, M)  - prekomputacja one-hot
#     expert0: uint8, shape (M,)
#     Zwraca: wektor dokładności dla całej populacji (P,) lub skalar gdy wejście było 1D
#     """
#     if pop.ndim == 1:
#         pop = pop[None, :]
#     P, R = pop.shape
#     M = expert0.shape[0]

#     # Liczymy głosy: tensordot po wymiarze reguł
#     # counts ma wymiar (P, 3, M) - liczba głosów na każdą klasę 0..2 w każdym punkcie
#     # Uwaga: rzutujemy na mały typ, żeby było lekko pamięciowo
#     counts = np.tensordot(pop.astype(np.int16), OH.astype(np.int16), axes=(1, 0))  # (P, 3, M)

#     # Predykcja to argmax po klasie (wiąże remisy na korzyść mniejszego indeksu: 0 > 1 > 2)
#     pred = np.argmax(counts, axis=1).astype(np.uint8)  # (P, M)
#     acc = (pred == expert0[None, :]).mean(axis=1)      # (P,)

#     # Osobnik bez żadnej reguły dostaje 0.0
#     empty = (pop.sum(axis=1) == 0)
#     if np.any(empty):
#         acc[empty] = 0.0

#     return acc[0] if acc.shape[0] == 1 else acc


# # ---------- SZYBKI PBIL W PEŁNI WEKTORYZOWANY ----------
# def pbil_batched(expert0, OH, dim, *,
#                  O1=0.1, O2=0.02, O3=0.05,
#                  pop_size=32, iters=200, seed=0,
#                  early_stop_patience=30):
#     """
#     - pop_size 32..64 działa zwykle dużo lepiej niż 3 i wciąż jest szybkie z batched ewaluacją
#     - early stopping zatrzyma się, gdy długo brak poprawy
#     """
#     rng = np.random.default_rng(seed)
#     prob = np.full(dim, 0.5, dtype=np.float32)

#     def sample_population(prob, size):
#         return (rng.random((size, prob.shape[0])) <= prob).astype(np.uint8)

#     pop = sample_population(prob, pop_size)
#     fit = fitness_population(pop, OH, expert0)

#     best_idx = int(np.argmax(fit))
#     best_x = pop[best_idx].copy()
#     best_f = float(fit[best_idx])
#     no_imp = 0

#     for _ in range(iters):
#         elite = pop[int(np.argmax(fit))].astype(np.float32)

#         # krok uczenia PBIL
#         prob = prob * (1.0 - O1) + elite * O1

#         # mutacja wektorowa
#         mut_mask = rng.random(dim) <= O2
#         if mut_mask.any():
#             rnd_bits = (rng.random(mut_mask.sum()) <= 0.5).astype(np.float32)
#             prob[mut_mask] = prob[mut_mask] * (1.0 - O3) + rnd_bits * O3

#         # stabilizacja
#         prob = np.clip(prob, 1e-3, 1.0 - 1e-3)

#         # nowa populacja i batched fitness
#         pop = sample_population(prob, pop_size)
#         fit = fitness_population(pop, OH, expert0)

#         idx = int(np.argmax(fit))
#         if fit[idx] > best_f:
#             best_f = float(fit[idx])
#             best_x = pop[idx].copy()
#             no_imp = 0
#         else:
#             no_imp += 1

#         if no_imp >= early_stop_patience:
#             break

#     return best_x.astype(np.uint8), best_f


# # ---------- PRZYKŁADOWE UŻYCIE ----------
# if __name__ == "__main__":
#     expert0, OH = load_data(("lista01/ClassificationRules.txt",
#                              "lista01/ImageExpertReduced.txt"))
#     R = OH.shape[0]  # liczba reguł, powinna wynosić 266

#     best_x, best_score = pbil_batched(expert0, OH, dim=R,
#                                       O1=0.1, O2=0.02, O3=0.05,
#                                       pop_size=48, iters=400, seed=42,
#                                       early_stop_patience=60)

#     print("Najlepsza dokładność:", best_score)
#     # Jeśli chcesz jeszcze raz potwierdzić:
#     # print("Check:", fitness_population(best_x, OH, expert0))


<div align="center" style="color: cyan">

# Zadanie 4

</div>

In [5]:
import numpy as np
import random
from typing import List

CHROMOSOME_LEN = 20
POPULATION_LEN = 3
ITERATION_NUM = 100

def binary_random(p):
    return 1 if random.random() <= p else 0

def weight_random():
    return random.random()

def change_weight(x, p):
    if random.random() <= p:
       return max(min(x + (0.1 * random.random()), 1.0), 0.0)
    else: 
        return max(min(x - (0.1 * random.random()), 1.0), 0.0)

def random_individual(p: List[float]) -> np.ndarray:
    return np.array([weight_random() for _ in p])

def new_individual(individual, p: List[float]) -> np.ndarray:
    return np.array([change_weight(individual[i], pi) for i, pi in enumerate(p)])

def random_population(p: List[float], N: int = POPULATION_LEN) -> np.ndarray:
    return np.array([random_individual(p) for _ in range(N)])

def new_population(population, p: List[float], N: int = POPULATION_LEN) -> np.ndarray:
    return np.array([new_individual(population[i], p) for i in range(N)])

def population_evaluation(P: List[List[float]], F):
    return np.array([F(x) for x in P])

def population_based_incremental_learning(F, N: int, O1, O2, O3):
    pro_vector = np.array([0.5 for _ in range(N)])                      # Initial Probabilty Vector
    ran_population = random_population(pro_vector, POPULATION_LEN)      # Initial Random Population 
    fitness = population_evaluation(ran_population, F)                  # Results
    best_individual = []
    best_score = 0
    for _ in range(ITERATION_NUM):
        x = ran_population[np.argmax(fitness)]

        if best_score < fitness[np.argmax(fitness)]:
            best_score = fitness[np.argmax(fitness)]
            best_individual = x

        for i in range(N):
            pro_vector[i] = pro_vector[i] * (1 - O1) + (x[i] * O1)

        for i in range(N):
            if random.random() <= O2:
                pro_vector[i] = pro_vector[i] * (1 - O3) + O3 * binary_random(0.5)

        ran_population = new_population(ran_population, pro_vector, POPULATION_LEN)
        fitness = population_evaluation(ran_population, F)

    return best_individual, best_score

def func(seq: List[float]) -> float:
    n = len(seq)
    m = len(classification[0])
    res = 0

    for p in range(m):
        c = Counter()
        for i in range(n):
            if seq[i] == 0:
                continue
            c[classification[i][p]] += seq[i]

        if c.most_common(1)[0][0] == expert[0][p]:
            res += 1

    return res / m


x, s = population_based_incremental_learning(func, 266, 0.1, 0.1, 0.1)
print(f'{x}: {(s * 100):.02f}')


[0.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 9.88373645e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00
 0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
 2.82737920e-02 1.00000000e+00 1.00000000e+00 1.97502068e-01
 0.00000000e+00 1.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00
 8.64374042e-01 9.15355822e-01 1.00000000e+00 1.00000000e+00
 9.17022802e-01 1.00000000e+00 1.00000000e+00 1.00000000e+00
 0.00000000e+00 0.00000000e+00 2.72946099e-01 3.43928018e-01
 4.08793583e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 8.87585931e-01 0.00000000e+00 9.32201852e-01 6.61708339e-01
 0.00000000e+00 2.86950140e-01 7.26113700e-01 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 7.66333374e-01 4.01792975e-02 1.00000000e+00
 5.84193324e-02 0.000000